<a href="https://colab.research.google.com/github/stavco9/datastreaming-final-project/blob/main/Scalable_and_Robust_Community_Detection_With_Randomized_Sketching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install networkx

In [ ]:
import random

In [ ]:
import math

#### Syntehetic Graph Generation

In [ ]:
import networkx as nx
import numpy as np

# Define the sizes of the blocks and the inter-block and intra-block edge probabilities
block_sizes = [10, 15, 20]  # 3 blocks with 10, 15, and 20 nodes
p_matrix = [[0.5, 0.1, 0.05],
            [0.1, 0.4, 0.07],
            [0.05, 0.07, 0.55]]

G = nx.stochastic_block_model(block_sizes, p_matrix)
A = nx.adjacency_matrix(G)
A_dense = A.todense()



### Sampling method 1:  URS
We will engineer the data to be cluster-balanced ($f = \frac{N}{n_{min}} = r$) to demonstrate, then show lower minimal-cluster-size values, before moving on to better sampling methods.


In [ ]:
def sample_URS(matrix,size):
    nodes = list(random.sample(range(matrix.shape[0]), size))
    return matrix[nodes, :][:, nodes]

In [ ]:
b= sample_URS(A_dense,4)

In [ ]:
b

array([[0, 1, 1, 0],
       [1, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 0, 0, 0]])

### Sampling method 2: SbS

‘Sparsity-based Sampling’ (SbS).

By capturing smaller clusters and producing more balanced
sketches, the clustering algorithm can be made even more
likely to succeed with the sketch than with the full matrix

In [ ]:
def sample_SbS(matrix,size):
    node_degrees = matrix.sum(axis=1)
    inverse_degrees = 1/node_degrees
    nodes = sample = random.choices(range(matrix.shape[0]), weights=inverse_degrees, k=size)
    return matrix[nodes, :][:, nodes]

In [ ]:
d = sample_SbS(A_dense,4)

In [ ]:
d

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

### Sampling method 3: SRS

randomized structure-preserving
sketching technique termed Spatial Random Sampling (SRS). With this approach, the individual edges of each node
are considered, and sampling is performed based on the
spatial distribution of the column vectors in the adjacency
matrix

In [ ]:
def sample_SRS(matrix,sample_size):

    sphere_sample = np.array([np.random.uniform(low=-1, high=1, size=matrix.shape[0]) for i in range(sample_size)])

    sampled_columns = []

    # Normalize

    row_sums_of_squares = np.sum(matrix ** 2, axis=1)
    normalized = matrix / row_sums_of_squares[:, np.newaxis]

    # Distance from randomly sampled points on the unit sphere
    Q = np.dot(sphere_sample, normalized)

    for i in range(sample_size):
        h = np.abs(Q[i, :])
        sampled_columns.append(np.argmax(h))


    return matrix[sampled_columns, :][:, sampled_columns]


In [ ]:
sample_SRS(A_dense,3)

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 0]])

### Spectral Clustering

In [ ]:
def laplacian(matrix):
    D = np.diag(np.sum(matrix, axis=1))
    L = D - matrix
    return L

In [ ]:
from sklearn.cluster import KMeans

def spectral_clustering(adjacency_matrix, num_clusters):
    # Compute the degree matrix
    degree_matrix = np.diag(adjacency_matrix.sum(axis=1))

    # Laplacian
    laplacian_matrix = degree_matrix - adjacency_matrix

    # Compute the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(laplacian_matrix)

    # Normalize eigenvectors row-wise to ensure equal length
    norm_eigenvectors = eigenvectors / np.linalg.norm(eigenvectors, axis=1, keepdims=True)

    # Use k-means to cluster the normalized eigenvectors
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(norm_eigenvectors)
    labels = kmeans.labels_

    # Group vertices by cluster
    clusters = {i: [] for i in range(num_clusters)}
    for i, label in enumerate(labels):
        clusters[label].append(i)

    return clusters

In [ ]:
spectral_clustering(A_dense,3)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{0: [0,
  3,
  5,
  8,
  9,
  10,
  13,
  14,
  15,
  16,
  20,
  26,
  27,
  28,
  29,
  30,
  34,
  37,
  38,
  41,
  42,
  43],
 1: [4, 6, 7, 11, 12, 17, 19, 21, 22, 23, 24, 25, 31, 35, 36, 39, 44],
 2: [1, 2, 18, 32, 33, 40]}

### Higher-Order Count Min Sketch

In [ ]:
def generate_hash_functions(D, k, seed=315870964):
    random.seed(seed)  # Ensure deterministic randomness
    salts = [random.randint(0, 2**32 - 1) for _ in range(D)]  # Unique salt for each hash function

    def hash_func_factory(salt):
        def hash_func(x):
            return (hash(x + salt) % k)
        return hash_func

    return [hash_func_factory(salt) for salt in salts]




In [ ]:
class HighOrderCountMinSketch:
    def __init__(self, delta, epsilon):
        self.width = int(math.ceil(2/epsilon))
        self.depth = int(math.ceil(np.log2(math.ceil(1/delta))))
        self.table = [[[0 for k in range(self.width)] for i in range(self.width)] for j in range(self.depth)]
        self.hash_functions = [generate_hash_functions(self.depth, self.width),generate_hash_functions(self.depth, self.width)]



    def add(self, item):
        for func in range(self.depth):
            self.table[func][self.hash_functions[0][func](item[0])][self.hash_functions[1][func](item[1])] += 1


    def estimate(self, item):
        estimates = []
        for func in range(self.depth):
            estimates.append(self.table[func][self.hash_functions[0][func](item[0])][self.hash_functions[1][func](item[1])])
        return min(estimates)





In [ ]:
hcms = HighOrderCountMinSketch(0.3, 0.5)

In [ ]:
stream = [[1,2],[3,5],[1,3],[1,2]]
for item in stream:
    hcms.add(item)

In [ ]:
measured = []
raw_errors = []
for i in range(len(stream)):
    if stream[i] in measured:
        continue
    else:
        measured.append(stream[i])
        raw_errors.append(hcms.estimate(stream[i])-stream.count(stream[i]))

In [ ]:
hcms.estimate([1,2])

2

### Generating random Graph Data

In [ ]:
def generate_random_graph(N, K):
    # Create an empty graph
    G = nx.Graph()

    # Add N nodes
    G.add_nodes_from(range(N))

    # List all possible edges
    possible_edges = [(i, j) for i in range(N) for j in range(i + 1, N)]

    # Randomly select K edges
    edges = random.sample(possible_edges, K)

    # Add edges to the graph
    G.add_edges_from(edges)

    return G

In [ ]:
list(generate_random_graph(10,3).edges())

[(4, 6), (6, 9), (7, 8)]

In [ ]:
hcms = HighOrderCountMinSketch(0.01, 0.05)

In [ ]:
stream = list(generate_random_graph(10000,100).edges())
for item in stream:
    hcms.add(item)

In [ ]:
measured = []
raw_errors = []
for i in range(len(stream)):
    if stream[i] in measured:
        continue
    else:
        measured.append(stream[i])
        raw_errors.append(hcms.estimate(stream[i])-stream.count(stream[i]))

In [ ]:
non_errors = []
for i in raw_errors:
    if i !=0:
        print(i)
    else:
        non_errors.append(i)

1
1
1
1
1
1
1
1
1
1


### Cluster-Graph Data

In [ ]:
block_sizes = [10, 15, 20]
p_matrix = [[0.5, 0.1, 0.05],
            [0.1, 0.4, 0.07],
            [0.05, 0.07, 0.55]]

G = nx.stochastic_block_model(block_sizes, p_matrix)

In [ ]:
hcms = HighOrderCountMinSketch(0.01, 0.05)

In [ ]:
stream = list(G.edges())
for item in stream:
    hcms.add(item)

In [ ]:
measured = []
raw_errors = []
for i in range(len(stream)):
    if stream[i] in measured:
        continue
    else:
        measured.append(stream[i])
        raw_errors.append(hcms.estimate(stream[i])-stream.count(stream[i]))

In [ ]:
non_errors = []
for i in raw_errors:
    if i !=0:
        print(i)
    else:
        non_errors.append(i)

1
1
1
1
1
1
1
1
1
1
1
1


### Using Sampling & Sketch on Real Data

In [ ]:
import bs4
import requests
import json
import os
from random import shuffle


### Community Detection

In [ ]:
from networkx.algorithms import community

block_sizes = [50, 50, 50]  # Example: 3 blocks with 10, 15, and 20 nodes, respectively
p_matrix = [[0.9, 0.1, 0.05],  # Probability of connections within and between blocks
            [0.1, 0.8, 0.02],
            [0.05, 0.02, 0.85]]

# Generate the stochastic block model graph
G = nx.stochastic_block_model(block_sizes, p_matrix)





### First, naively trying the entire graph:

In [ ]:
communities_generator = community.girvan_newman(G)

In [ ]:
%time top_level_communities = next(communities_generator)

CPU times: user 50.8 s, sys: 177 ms, total: 51 s
Wall time: 51 s


In [ ]:
communities = sorted(map(sorted, top_level_communities))
print("Communities:", communities)


Communities: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]]


## Using URS:

In [ ]:
communities_generator = community.girvan_newman(nx.from_numpy_array(sample_URS(nx.adjacency_matrix(G).todense(),size = 50)))

In [ ]:
%time top_level_communities = next(communities_generator)

CPU times: user 301 ms, sys: 0 ns, total: 301 ms
Wall time: 301 ms


In [ ]:
communities = sorted(map(sorted, top_level_communities))
print("Communities:", communities)


Communities: [[0, 2, 3, 4, 5, 6, 7, 9, 10, 11, 13, 15, 16, 17, 18, 21, 23, 25, 26, 27, 30, 32, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49], [1, 8, 12, 14, 19, 20, 22, 24, 28, 29, 31, 33, 34, 36, 37, 47]]


### Using SbS:

In [ ]:
communities_generator = community.girvan_newman(nx.from_numpy_array(sample_SbS(nx.adjacency_matrix(G).todense(),size = 50)))

In [ ]:
%time top_level_communities = next(communities_generator)

CPU times: user 137 ms, sys: 67 µs, total: 137 ms
Wall time: 137 ms


In [ ]:
communities = sorted(map(sorted, top_level_communities))
print("Communities:", communities)


Communities: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 44, 46, 47, 49], [13, 15, 17, 19, 20, 40, 42, 43, 45, 48]]


### Using SRS:

In [ ]:
communities_generator = community.girvan_newman(nx.from_numpy_array(sample_SRS(nx.adjacency_matrix(G).todense(),sample_size = 50)))

In [ ]:
%time top_level_communities = next(communities_generator)

CPU times: user 623 ms, sys: 581 ms, total: 1.2 s
Wall time: 537 ms


In [ ]:
communities = sorted(map(sorted, top_level_communities))
print("Communities:", communities)


Communities: [[0, 2, 3, 4, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 29, 30, 31, 32, 34, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49], [1, 5, 6, 7, 8, 9, 10, 16, 25, 28, 33, 35, 40]]


We can see how even with only a third of the original graph nodes, SRS is a good preserver of the graph structure!

### Blog Corpus

Here we need a valid Kaggle API key to download the data. My own key is submitted as a .json file together with this notebook.

In [ ]:
!pip install kaggle

In [ ]:
import os
# Make .kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)
# Copy the kaggle.json to this directory
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d rtatman/blog-authorship-corpus


blog-authorship-corpus.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip blog-authorship-corpus.zip


Archive:  blog-authorship-corpus.zip
replace blogtext.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd

In [ ]:
blogs_ds = pd.read_csv("/content/blogtext.csv")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


# Assuming 'df' is your DataFrame and it has a column 'text' with the texts
texts = blogs_ds.head(10)['text'].values

vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(texts)

# Transposing the result to get words as rows and texts as columns
binary_matrix = X.T.toarray()

# Creating a DataFrame for better readability, with words as the index
words = vectorizer.get_feature_names_out()
binary_df = pd.DataFrame(binary_matrix, index=words)

binary_df


,0,1,2,3,4,5,6,7,8,9
000,0,0,0,0,0,1,0,0,1,0
07,0,0,1,0,0,0,0,0,0,0
10,0,0,1,0,0,1,0,0,0,0
100,1,0,1,0,0,0,0,0,0,0
1000,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
your,0,0,1,0,0,1,0,0,1,0
yourself,0,0,1,0,0,0,1,0,0,0
zero,0,0,0,0,0,1,0,0,0,0
zone,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Discarding all words appearing in only 1 document
filtered_df = binary_df[binary_df.sum(axis=1) > 1]

In [ ]:
filtered_df

,0,1,2,3,4,5,6,7,8,9
000,0,0,0,0,0,1,0,0,1,0
10,0,0,1,0,0,1,0,0,0,0
100,1,0,1,0,0,0,0,0,0,0
1950,0,0,0,0,0,0,0,1,1,0
20,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
years,0,0,0,0,0,0,0,1,1,0
yes,0,0,1,0,0,0,0,1,0,0
you,0,0,1,0,1,1,1,0,1,0
your,0,0,1,0,0,1,0,0,1,0


In [ ]:
adjacency_matrix = filtered_df.dot(filtered_df.T)

# Convert the adjacency matrix to binary (1 if words co-occur, 0 otherwise)
adjacency_matrix = adjacency_matrix.applymap(lambda x: 1 if x > 0 else 0)

adjacency_matrix


,000,10,100,1950,20,24,400,50,600,90,...,won,wonder,work,world,would,years,yes,you,your,yourself
000,1,1,0,1,1,1,1,1,1,1,...,1,1,0,0,1,1,0,1,1,0
10,1,1,1,0,1,0,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
100,0,1,1,0,1,0,1,1,0,1,...,1,1,1,1,1,0,1,1,1,1
1950,1,0,0,1,0,1,0,1,1,0,...,0,1,0,1,1,1,1,1,1,0
20,1,1,1,0,1,0,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
years,1,0,0,1,0,1,0,1,1,0,...,0,1,0,1,1,1,1,1,1,0
yes,0,1,1,1,1,0,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
you,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
your,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


### First, naively trying the entire graph:

In [ ]:
communities_generator = community.girvan_newman(nx.from_numpy_array(adjacency_matrix.to_numpy()))

In [ ]:
%time top_level_communities = next(communities_generator)

CPU times: user 1min 40s, sys: 289 ms, total: 1min 40s
Wall time: 1min 40s


In [ ]:
communities = sorted(map(sorted, top_level_communities))
print("Communities:", communities)


Communities: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

### Sampling method 1:  URS

In [ ]:
communities_generator = community.girvan_newman(nx.from_numpy_array(sample_URS(adjacency_matrix.to_numpy(),size = 100)))

In [ ]:
%time top_level_communities = next(communities_generator)

CPU times: user 6.02 s, sys: 25.6 ms, total: 6.05 s
Wall time: 6.04 s


In [ ]:
communities = sorted(map(sorted, top_level_communities))
print("Communities:", communities)


Communities: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], [15]]


### SbS

In [ ]:
communities_generator = community.girvan_newman(nx.from_numpy_array(sample_SbS(adjacency_matrix.to_numpy(),size = 100)))

In [ ]:
%time top_level_communities = next(communities_generator)

CPU times: user 1.4 s, sys: 3.91 ms, total: 1.4 s
Wall time: 1.4 s


In [ ]:
communities = sorted(map(sorted, top_level_communities))
print("Communities:", communities)


Communities: [[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], [4]]


### SRS

In [ ]:
communities_generator = community.girvan_newman(nx.from_numpy_array(sample_SRS(adjacency_matrix.to_numpy(),sample_size = 100)))

In [ ]:
%time top_level_communities = next(communities_generator)

CPU times: user 5.85 s, sys: 500 ms, total: 6.35 s
Wall time: 5.79 s


In [ ]:
communities = sorted(map(sorted, top_level_communities))
print("Communities:", communities)


Communities: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], [37]]


### High Order CountMinSketch:

In [ ]:
hcms = HighOrderCountMinSketch(0.03, 0.05)

In [ ]:
stream = list(G.edges(nx.from_numpy_array(adjacency_matrix.to_numpy())))
for item in stream:
    hcms.add(item)

In [ ]:
measured = []
raw_errors = []
relative_error = []
for i in range(len(stream)):
    if stream[i] in measured:
        continue
    else:
        measured.append(stream[i])
        raw_errors.append(hcms.estimate(stream[i])-stream.count(stream[i]))
        relative_error.append((hcms.estimate(stream[i])-stream.count(stream[i]))/stream.count(stream[i]))


In [ ]:
sum(raw_errors)/len(stream)

1.9235064209938582

In [ ]:
sum(relative_error)/len(stream)

1.9235064209938582

### Wikipedia Articles

In [ ]:
!pip install beautifulsoup4


In [ ]:
import requests
from bs4 import BeautifulSoup

links_base_url = 'https://en.wikipedia.org/wiki/'
links_uri = 'Wikipedia:Good_articles/By_length'

# Fetch the page
response = requests.get(f"{links_base_url.rstrip('/')}/{links_uri}",headers={'User-Agent': 'Mozilla/5.0'})
soup = bs4.BeautifulSoup(response.text,'html.parser')
table = soup.find('table')

# Extract links; this also might need refinement based on the table's structure
links = [a['href'] for a in table.find_all('a', href=True) if a['href'].startswith('/wiki/')]




In [ ]:
def download_wikipedia_article(title):
    title_quoted = requests.utils.quote(title.replace(" ", "_"))
    url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&titles={title_quoted}&redirects=true"
    response = requests.get(url)
    data = response.json()
    page = next(iter(data['query']['pages'].values()))
    content = page.get("extract", "")
    return {"title": title, "content": content}

# List to store article dictionaries
articles_data = []

for link in links[:50]:  # Adjust the range as needed for demonstration
    title = link.split("/wiki/")[-1].replace("_", " ")
    article_dict = download_wikipedia_article(title)
    articles_data.append(article_dict)

# Convert list of dictionaries to DataFrame
df_articles = pd.DataFrame(articles_data)

In [ ]:
texts = df_articles['content'].values

vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(texts)

# Transposing the result to get words as rows and texts as columns
binary_matrix = X.T.toarray()

# Creating a DataFrame for better readability, with words as the index
words = vectorizer.get_feature_names_out()
binary_df = pd.DataFrame(binary_matrix, index=words)

# Discarding all words appearing in only 1 document
filtered_df = binary_df[binary_df.sum(axis=1) > 1]

adjacency_matrix = filtered_df.dot(filtered_df.T)

# Convert the adjacency matrix to binary (1 if words co-occur, 0 otherwise)
adjacency_matrix = adjacency_matrix.applymap(lambda x: 1 if x > 0 else 0)



In [ ]:
hcms = HighOrderCountMinSketch(0.01, 0.01)

In [ ]:
stream = list(G.edges(nx.from_numpy_array(sample_SRS(adjacency_matrix.to_numpy(),sample_size = 100))))
for item in stream:
    hcms.add(item)

In [ ]:
measured = []
raw_errors = []
relative_error = []
for i in range(len(stream)):
    if stream[i] in measured:
        continue
    else:
        measured.append(stream[i])
        raw_errors.append(hcms.estimate(stream[i])-stream.count(stream[i]))
        relative_error.append((hcms.estimate(stream[i])-stream.count(stream[i]))/stream.count(stream[i]))
